In [ ]:
import numpy as np
import pandas as pd

Proportional-integral control tracks the cumulative sum of the differences between setpoint and the process variable. 

implement simple control strategies where the manipulated variable depends only on currant values of the process

def proportional(PV, SP):
    MV = Kp * (SP - PV)
    return MV

In [2]:
# Kp = specific gain
# SP = setpoint
# PV = process variable
# MV = manipulated variable

# when set a current value of the PV, the controllers will return a value for the MV
def proprotional (Kp, SP):
    """creates proportional controllers with specific gain and setpoint
    """
    MV = 0
    while True:
        PV = yield MV
        MV = Kp*(SP - PV)

In [3]:
# PID control implementation

def PID(Kp, Ki, Kd, MV_bar = 0):
    # initialize stored data
    e_prev = 0
    t_prev = -100
    I = 0

    # initial control
    MV = MV_bar

    while True:
        # yield MV, wait for new t, PV, SP
        t, PV, SP = yield MV

        # PID calculations
        e = SP - PV

        P = Kp*e
        I = I + Ki * e * (t - t_prev)
        D = Kd * (e - e_prev)/(t - t_prev)

        MV = MV_bar + P + I + D

        # update stored data for next iteration
        e_prev = e
        t_prev = t

In [4]:
import time

def _clamp(value, limits):
    lower, upper = limits
    if value is None:
        return None
    elif (upper is not None) and (value > upper):
        return upper
    elif (lower is not None) and (value < lower):
        return lower
    return value



class PID:
    """PID Controller
    """

    def __init__(self, P=0.2, I=0.0, D=0.0, current_time=None):

        self.Kp = P
        self.Ki = I
        self.Kd = D

        self.sample_time = 0.00
        self.current_time = current_time if current_time is not None else time.time()
        self.last_time = self.current_time

        self.clear()

    def clear(self):
        """Clears PID computations and coefficients"""
        self.SetPoint = 0.0

        self.PTerm = 0.0
        self.ITerm = 0.0
        self.DTerm = 0.0
        self.last_error = 0.0

        # Windup Guard
        self.int_error = 0.0
        self.windup_guard = 20.0

        self.output = 0.0

    def update(self, feedback_value, current_time=None):
        """Calculates PID value for given reference feedback
        .. math::
            u(t) = K_p e(t) + K_i \int_{0}^{t} e(t)dt + K_d {de}/{dt}
        .. figure:: images/pid_1.png
           :align:   center
           Test PID with Kp=1.2, Ki=1, Kd=0.001 (test_pid.py)
        """
        error = self.SetPoint - feedback_value

        self.current_time = current_time if current_time is not None else time.time()
        delta_time = self.current_time - self.last_time
        delta_error = error - self.last_error

        if (delta_time >= self.sample_time):
            self.PTerm = self.Kp * error
            self.ITerm += error * delta_time

            if (self.ITerm < -self.windup_guard):
                self.ITerm = -self.windup_guard
            elif (self.ITerm > self.windup_guard):
                self.ITerm = self.windup_guard

            self.DTerm = 0.0
            if delta_time > 0:
                self.DTerm = delta_error / delta_time

            # Remember last time and last error for next calculation
            self.last_time = self.current_time
            self.last_error = error

            self.output = self.PTerm + (self.Ki * self.ITerm) + (self.Kd * self.DTerm)

    def setKp(self, proportional_gain):
        """Determines how aggressively the PID reacts to the current error with setting Proportional Gain"""
        self.Kp = proportional_gain

    def setKi(self, integral_gain):
        """Determines how aggressively the PID reacts to the current error with setting Integral Gain"""
        self.Ki = integral_gain

    def setKd(self, derivative_gain):
        """Determines how aggressively the PID reacts to the current error with setting Derivative Gain"""
        self.Kd = derivative_gain

    def setWindup(self, windup):
        """Integral windup, also known as integrator windup or reset windup,refers to the situation in a PID feedback controller where
        a large change in setpoint occurs (say a positive change)and the integral terms accumulates a significant error
        during the rise (windup), thus overshooting and continuing to increase as this accumulated error is unwound
        (offset by errors in the other direction). 
        The specific problem is the excess overshooting.
        """
        self.windup_guard = windup

    def setSampleTime(self, sample_time):
        """PID that should be updated at a regular interval.
        Based on a pre-determined sampe time, the PID decides if it should compute or return immediately.
        """
        self.sample_time = sample_time

In [7]:
pid = PID()

TypeError: 'int' object is not callable

In [4]:
#import PID
import time
import os.path

from OmegaExpansion import AdcExp
from OmegaExpansion import pwmExp

pwmExp.setVerbosity(-1)
pwmExp.driverInit()
adc = AdcExp.AdcExp()

targetT = 35
P = 10
I = 1
D = 1

pid = PID.PID(P, I, D)
pid.SetPoint = targetT
pid.setSampleTime(1)

def readConfig ():
	global targetT
	with open ('/tmp/pid.conf', 'r') as f:
		config = f.readline().split(',')
		pid.SetPoint = float(config[0])
		targetT = pid.SetPoint
		pid.setKp (float(config[1]))
		pid.setKi (float(config[2]))
		pid.setKd (float(config[3]))

def createConfig ():
	if not os.path.isfile('/tmp/pid.conf'):
		with open ('/tmp/pid.conf', 'w') as f:
			f.write('%s,%s,%s,%s'%(targetT,P,I,D))

createConfig()

while 1:
	readConfig()
	#read temperature data
	a0 = adc.read_voltage(0)
	temperature = (a0 - 0.5) * 100

	pid.update(temperature)
	targetPwm = pid.output
	targetPwm = max(min( int(targetPwm), 100 ),0)

	print ("Target: %.1f C | Current: %.1f C | PWM: %s %% ")% (targetT, temperature, targetPwm)

	# Set PWM expansion channel 0 to the target setting
	pwmExp.setupDriver(0, targetPwm, 0)
	time.sleep(0.5)

ModuleNotFoundError: No module named 'OmegaExpansion'

In [ ]:
# load PID parameters and constants:

# PID Parameters
Kc = 6.0
tauI = 75.0 # sec
tauD = 0.0 # sec

# PID coeff in terms of tuning parameters
KP = Kc
KI = Kc / tauI
KD = Kc * tauD

# OPbias for controller (initial heater)
op0 = 0

# upper and lower bounds on heater level
ophi = 100
oplo = 0

In [ ]:
# calculate the error between the setpoint (sp) and the process variable (pv)

error = sp - pv

# calculate the integral error
# the integral portion is calculater numerically, adding whatever the previous error to the current time step error:

ierr = ierr + KI * error * dt

# the derivative portion of the controller
dpv = (pv - pv_last) / dt

## all together
# calculate the PID output

P = KP * error
I = ierr
D = -KD * dpv
op = op0 + P + I + D

# add a layer to account for the general limits
if op < oplo or op > ophi:
    I = I - KI * error * dt
    # clip output
    op = max(oplo, min(ophi,op))

In [ ]:
# Generating data from the PID controller

# run time in minutes
run_time = 90

# number of cycles
loops = int(60*run_time)

# arrays fo storing data
variable1 = np.zeros(loops) # measured variable
error_sp = np.zeros(loops) # setpoint error
variable2 = np.zeros(loops) # other variables
tm = np.zeros(loops) # time

# Measures variable set point from controlmachines.cloud
measured_value

# vary measured variable setpoint
end = 30 # leave the 1st 30 second of measures variable setpoint as measured
while end <= loops:
    start = end
    # keep new value as setpoint for 4 to 10 min
    end += random.randint(240, 600) # sec
    measured_value[start:end] = random.randint(30,70)

Second part 

After the LSTM simulations, here the LSTM neyral network is implemented to emulate the PID controller behavior

Setup:

Define two variables arrays to store the PID output and LSTM output;
The loop controller with LSTM is the same from the PID controller;


In [ ]:
# set up run

## import model and parameters from LSTM

s_x = model_params['Xscale']
s_y = model_params['Yscale']
window = model_params['window']

# run time in minutes
run_time = 30

# number of cycles
loop = int(60*run_time) # but mine time steps are in minutes

# array for storing the data
variable1 = np.zeros(loop) # measures variable
variable2_PID = np.zeros(loop) #  other variable for PID controller
variable2_lstm = np.zeros(loop) #  other variable for LSTM controller
tm = np.zeros(loop) # time

# variable 1 set point from controlmachines.clound
measured_value

# vary measure variable set point
# leave 1st window + 15 seconds of temperature set point as room temperature
end = window + 15
while end <= loop:
    start = end
    # keep new set point value for 4 to 10 min
    end += random.randint(240,600)
    measured_value[start:end] = random.randint(30,70)
